In [1]:
import numpy as np
import csv as csv
import json    
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import biosppy.signals.resp as resp
# Returns
#    -------
#    ts : array
#        Signal time axis reference (seconds).
#    filtered : array
#        Filtered Respiration signal.
#    zeros : array
#        Indices of Respiration zero crossings.
#    resp_rate_ts : array
#        Respiration rate time axis reference (seconds).
#    resp_rate : array
#        Instantaneous respiration rate (Hz).

In [3]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

### Read header

In [4]:
# fname = 'https://github.com/malfarasplux/biofeatures/tree/master/riot_rip_acquisition/ak/opensignals_192.168.1.1008888_2020-02-05_11-54-42.txt'
fname = '../riot_rip_acquisition/ak/opensignals_192.168.1.1008888_2020-02-05_11-54-42.txt'
with open(fname) as datafile:
    readdata = csv.reader(datafile, delimiter='\t')
    header = [next(readdata), next(readdata), next(readdata)]
    header_json = json.loads(header[1][0][1:])
print(header_json)

{'192.168.1.100:8888': {'sensor': ['RIOT', 'RIOT', 'RIOT', 'RIOT', 'RIOT'], 'device name': '192.168.1.100:8888', 'column': ['nSeq', 'DI', 'ACCx', 'ACCy', 'ACCz', 'ANALOG1', 'ANALOG2'], 'sync interval': 2, 'time': '11:54:43.221', 'comments': '', 'device connection': '192.168.1.100:8888', 'channels': [1, 2, 3, 13, 14], 'keywords': '', 'convertedValues': 0, 'mode': 0, 'digital IO': [0, 1], 'firmware version': 'riot', 'device': 'riot', 'position': 0, 'sampling rate': 200, 'label': ['ACCx', 'ACCy', 'ACCz', 'ANALOG1', 'ANALOG2'], 'resolution': [1, 1, 1, 1, 1], 'date': '2020-2-5', 'special': [{}, {}, {}, {}, {}]}}


### Read data

In [5]:
data = np.genfromtxt(fname)
print(data.shape)

(48150, 7)


In [6]:
# Locate the sampling rate 
srate = header_json[list(header_json.keys())[0]]['sampling rate']
srate

200

In [7]:
# Locate the analog channel
col_num  = header_json[list(header_json.keys())[0]]['column'].index('ANALOG1')
col_num

5

In [8]:
ripdata = data[:,col_num]

### Test the library

In [9]:
import biofeatures

In [10]:
B = biofeatures.breathing(ripdata[:2000])

In [11]:
B.resp_intervals()

D:\Anaconda37\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [12]:
B.resp_features()

In [13]:
B.features

{'breath_avg_len': 3.33,
 'inhale_duration': 5.885,
 'exhale_duration': 4.105,
 'inhale_exhale_ratio': 1.4336175395858708}

In [14]:
B.interval_breathe_in

[True, False, True, False, True]

In [15]:
B.interval_lengths

array([247, 402, 455, 419, 475], dtype=int64)

In [16]:
np.sum(B.interval_lengths[B.interval_breathe_in])/B.srate

5.885

In [17]:
np.sum(B.interval_lengths[np.logical_not(B.interval_breathe_in)])/B.srate

4.105

In [18]:
B.feature_names

dict_keys(['breath_avg_len', 'inhale_duration', 'exhale_duration', 'inhale_exhale_ratio'])

In [19]:
B.resp_rate*60

array([15.18987342])

### Simulate real time

In [20]:
import time

In [21]:
ti = time.time()

In [22]:
tf = time.time()
ind = 10000
ts = 0.5

# 5 seconds, every 0.5
while tf-ti < 5:
    B.data = ripdata[ind-B.buffer_length:ind]
    ind += int(ts*B.srate)
    B.resp_intervals()
    B.resp_features()
    print(B.features)
    time.sleep(ts)
    tf = time.time()


{'breath_avg_len': 2.495, 'inhale_duration': 3.34, 'exhale_duration': 1.65, 'inhale_exhale_ratio': 2.0242424242424244}
{'breath_avg_len': 1.6633333333333333, 'inhale_duration': 3.17, 'exhale_duration': 1.82, 'inhale_exhale_ratio': 1.7417582417582418}
{'breath_avg_len': 2.495, 'inhale_duration': 3.07, 'exhale_duration': 1.92, 'inhale_exhale_ratio': 1.5989583333333333}
{'breath_avg_len': 2.495, 'inhale_duration': 2.59, 'exhale_duration': 2.4, 'inhale_exhale_ratio': 1.0791666666666666}
{'breath_avg_len': 2.495, 'inhale_duration': 1.77, 'exhale_duration': 3.22, 'inhale_exhale_ratio': 0.5496894409937888}
{'breath_avg_len': 2.495, 'inhale_duration': 1.915, 'exhale_duration': 3.075, 'inhale_exhale_ratio': 0.6227642276422765}
{'breath_avg_len': 1.6633333333333333, 'inhale_duration': 1.93, 'exhale_duration': 3.06, 'inhale_exhale_ratio': 0.630718954248366}
{'breath_avg_len': 1.6633333333333333, 'inhale_duration': 2.65, 'exhale_duration': 2.34, 'inhale_exhale_ratio': 1.1324786324786325}
{'breath_

In [23]:
B.buffer_length


1000

In [24]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import time


In [25]:
riot_ip = '192.168.0.102'
riot_port = 31000
actuator_port = 12000
actuator_ip = '192.168.0.103'

client = SimpleUDPClient(riot_ip, riot_port) 
resp_data = []

In [26]:
def process_riot_data(unused_addr, *values):
    global resp_data, last_update, client
    global t0, B
    
        
    new_data = values[12]
    resp_data.append(new_data)
    print(new_data)

    warmup = 10
    tf = time.time()

#     dmax = 15
#     if tf-ti > 0.5 and tf-t0 < warmup+dmax and tf-t0 > warmup: 
#         print(tf-t0)
#         real_time_feat()
#         ti = time.time()


    if tf-t0 > warmup: 
        print("WARMUP")
        if not B.WU:
            B.WU = True
            B.update_loop()

    
#     if len(resp_data) > 200*10 and time.time() - last_update > update_freq:
#         last_int, breathe_in = biofeatures.calc_resp_intervals(resp_data, last_breath = True)
                
#         if breathe_in:
#             print("Breathing in")
#             client.send_message("/actuator/inflate", 100.0)
#         else:
#             print("Breathing out")
#             client.send_message("/actuator/inflate", -100.0)
        
#         last_update = time.time()
    
    # only save the last 5 min of data
    if len(resp_data) > 200 * 60 * 5:
        resp_data = resp_data[-200*60*5:]

In [27]:
def real_time_feat():
    global B, resp_data
    ts = 0.5
    B.data = resp_data[-B.buffer_length:]
    B.resp_intervals()
    B.resp_features()
    print(B.features)

In [ ]:
t0 = time.time()
ti = time.time()
riot_dispatcher = dispatcher.Dispatcher()
riot_dispatcher.map("/*/raw", process_riot_data)

server = osc_server.ThreadingOSCUDPServer((riot_ip, riot_port), riot_dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

Serving on ('192.168.0.102', 31000)


In [ ]:
B.WU

In [ ]:
tf-t0